In [1]:
# This imports the Metadata
# I named the given test file "TEST-Official-Metadata-MN.csv" for an example
# Change the csv name to reflect what the metadate file is called
import pandas
df = pandas.read_csv('TEST-Official-Metadata-MN.csv',dtype=object)
import numpy as np  # Import numpy for NaN representation
df = df.replace(np.nan, '')

df

,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,birth_date,...,adi_natrank,adi_staternk,county,zip,race,raceth,hispanic,gender,time_since,age_group
0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,test,...,25,2,Hennepin County,55406,test,test,test,test,test,test


In [2]:
# This creates a new column of assigned or unassigned USDA urban/rural classifications
# With the Wisconsin metadata, some samples did not have a Census Tract number, which is why this is necessary to show that some samples will remain unassigned
# Thus, this is not necessary if all your samples have a Census Tract Number
# This also provides clarification for the blank values
def assigned_or_unassigned(row):
    
    # This is testing the following: if census_tract is Nan; if not, there is a confirmed rural/urban classification
    if pandas.notna(row['census_tract']) and any(char.isdigit() for char in str(row['census_tract'])):
        print('Assigned')
        return 'Assigned'
    else:
        print('Unassigned')
        return 'Unassigned'

df['Assigned-or-Unassigned-USDA-Classification'] = df.apply(assigned_or_unassigned, axis=1)

df

Assigned


,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,birth_date,...,adi_staternk,county,zip,race,raceth,hispanic,gender,time_since,age_group,Assigned-or-Unassigned-USDA-Classification
0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,test,...,2,Hennepin County,55406,test,test,test,test,test,test,Assigned


In [3]:
# Here we read in the file from the USDA classification
# I titled the RUCA file as "MN-RUCA-Definitions"
ruca = pandas.read_csv('MN-RUCA-Definitions.csv',dtype=object)

ruca

,State-County FIPS Code,Select State,Select County,Total-FIPS-Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,27001,MN,Aitkin County,27001770100,10,10,"2,327",458.9,5.1
1,27001,MN,Aitkin County,27001770200,10,10,"2,336",461.8,5.1
2,27001,MN,Aitkin County,27001770300,10,10,"3,268",33.4,97.8
3,27001,MN,Aitkin County,27001770400,10,10,"3,052",460.1,6.6
4,27001,MN,Aitkin County,27001790501,10,10,"1,957",294.3,6.7
...,...,...,...,...,...,...,...,...,...
1333,27171,MN,Wright County,27171101300,7,7,"4,964",69.1,71.9
1334,27173,MN,Yellow Medicine County,27173970100,7,7,"3,371",147.5,22.9
1335,27173,MN,Yellow Medicine County,27173970200,10,10,"2,520",144.1,17.5
1336,27173,MN,Yellow Medicine County,27173970300,10,10,"2,517",286.3,8.8


In [4]:
# Merge the data from both files
# We are matching the values from the 11 digit census_tract with the 11 digit Total-FIPS-Code from the USDA
# See the specified column names from the example in Git
usda_merged_df = pandas.merge(df, ruca, left_on='census_tract', right_on='Total-FIPS-Code', how='left')

# This prevents Unassigned values from being dropped
usda_merged_df.fillna(value={'column_name_in_ruca': 'Unassigned'}, inplace=True)

# This removes NaN values
usda_merged_df = usda_merged_df.replace(np.nan, '')

usda_merged_df

,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,birth_date,...,Assigned-or-Unassigned-USDA-Classification,State-County FIPS Code,Select State,Select County,Total-FIPS-Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,test,...,Assigned,27053,MN,Hennepin County,27053106400,1,1,"1,821",0.2,"8,860.50"


In [5]:
# This creates a new file that I titled "MN-Midwest-Sars-Cov-2-ASSIGNED-Metadata"
usda_merged_df.to_csv('MN-Midwest-Sars-Cov-2-ASSIGNED-Metadata.csv')

In [6]:
# This creates a new column that categorizes urban or rural based on USDA data
# The function also determines Urban or Rural classification
file_path = 'MN-Midwest-Sars-Cov-2-ASSIGNED-Metadata.csv'
df = pandas.read_csv(file_path)
def urban_or_rural_classification(row):
    primary_ruca_code = row['Primary RUCA Code 2010']
    
    if primary_ruca_code in [1, 2, 3, 4, 5, 6, 7,8]:
        return 'Urban'
    elif primary_ruca_code in [9, 10]:
        return 'Rural'
    else:
        return 'Unknown'

# Create the new column
df['Urban-or-Rural-USDA-Classification'] = df.apply(urban_or_rural_classification, axis=1)

#This removes NaN values
df = df.replace(np.nan, '')

# Save the modified DataFrame back to the CSV file
df.to_csv(file_path, index=False)

df

,Unnamed: 0,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,...,State-County FIPS Code,Select State,Select County,Total-FIPS-Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",Urban-or-Rural-USDA-Classification
0,0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,...,27053,MN,Hennepin County,27053106400,1,1,"1,821",0.2,"8,860.50",Urban


In [7]:
# Here we read in the file from the CENSUS classification
import pandas
census = pandas.read_csv('NHGIS-CensusTract-Data-MN.csv',dtype=object)

census

,GISJOIN,YEAR,GEOID,GEOCODE,STATE,COUNTY,COUNTYA,TRACTA,U71001,U71002,U71003,UrbanThreshold,RuralThreshold
0,GIS Join Match Code,Data File Year,Geographic Record Identifier,Geographic Code Identifier,STATE (FIPS) Name,County (FIPS) Name,County (FIPS) Name,CensusTractCode,Total,Urban,Rural,UrbanThreshold,RuralThreshold
1,G2700010770100,2020,1400000US27001770100,27001770100,Minnesota,Aitkin County,1,770100,2244,0,2244,0,100
2,G2700010770200,2020,1400000US27001770200,27001770200,Minnesota,Aitkin County,1,770200,2278,0,2278,0,100
3,G2700010770300,2020,1400000US27001770300,27001770300,Minnesota,Aitkin County,1,770300,3298,0,3298,0,100
4,G2700010770401,2020,1400000US27001770401,27001770401,Minnesota,Aitkin County,1,770401,1308,0,1308,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,G2701710101300,2020,1400000US27171101300,27171101300,Minnesota,Wright County,171,101300,5154,0,5154,0,100
1502,G2701730970100,2020,1400000US27173970100,27173970100,Minnesota,Yellow Medicine County,173,970100,2999,0,2999,0,100
1503,G2701730970200,2020,1400000US27173970200,27173970200,Minnesota,Yellow Medicine County,173,970200,2362,0,2362,0,100
1504,G2701730970300,2020,1400000US27173970300,27173970300,Minnesota,Yellow Medicine County,173,970300,2302,0,2302,0,100


In [8]:
usda_merged_df = df

usda_merged_df

,Unnamed: 0,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,...,State-County FIPS Code,Select State,Select County,Total-FIPS-Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",Urban-or-Rural-USDA-Classification
0,0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,...,27053,MN,Hennepin County,27053106400,1,1,"1,821",0.2,"8,860.50",Urban


In [9]:
df['census_tract'] = df['census_tract'].astype(str)

# Merge based on different column names
merged_df = pandas.merge(df, census, left_on='census_tract', right_on='GEOCODE', how='left')

#This prevents Unassigned values from being dropped
merged_df['GEOCODE'].fillna(value='Unassigned', inplace=True)

#This removes NaN values
merged_df.replace(np.nan, '', inplace=True)

merged_df

,Unnamed: 0,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,last_name,...,GEOCODE,STATE,COUNTY,COUNTYA,TRACTA,U71001,U71002,U71003,UrbanThreshold,RuralThreshold
0,0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,test,...,27053106400,Minnesota,Hennepin County,53,106400,1817,1817,0,100,0


In [10]:
merged_df.to_csv('MN-Combined-Assigned-Metadata.csv')

In [11]:
# This creates a new column that vaugly categorizes urban or rural based on Census data

file_path = 'MN-Combined-Assigned-Metadata.csv'
df = pandas.read_csv(file_path)

df['Urban-or-Rural-Census-Classification'] = ''

# Define conditions and update the 'Urban_or_Rural' column
df.loc[(df['UrbanThreshold'] > 50) & (df['RuralThreshold'] < 50), 'Urban-or-Rural-Census-Classification'] = 'Urban'
df.loc[(df['RuralThreshold'] > 50) & (df['UrbanThreshold'] < 50), 'Urban-or-Rural-Census-Classification'] = 'Rural'

#This removes NaN values
df = df.replace(np.nan, '')

# Save the modified DataFrame back to the CSV file
df.to_csv(file_path, index=False)

df

,Unnamed: 0.1,Unnamed: 0,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,...,STATE,COUNTY,COUNTYA,TRACTA,U71001,U71002,U71003,UrbanThreshold,RuralThreshold,Urban-or-Rural-Census-Classification
0,0,0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,...,Minnesota,Hennepin County,53,106400,1817,1817,0,100,0,Urban


In [12]:
# Further clean up column headers, removing spaces and other punctuation 
df.columns = df.columns.str.replace(r'[\(\)]', '').str.replace(r'[\s\W]', '_', regex=True)
# Further clean up column headers, removing spaces and other punctuation 
df = df.rename(columns={'Secondary RUCA Code, 2010 (see errata)': 'Secondary RUCA Code 2010', 
                        'Land Area (square miles), 2010' : 'Land Area sq miles 2010', 
                        'Population Density (per square mile), 2010' : 'Pop Density per sq mile 2010'
                       })
df.columns = df.columns.str.replace(r'[\(\)]', '').str.replace(r'[\s\W]', '_', regex=True)

df

,Unnamed__0_1,Unnamed__0,case_id,covid_mn_id,covid_virus_name,gisaid_id,genbank_id,ncbi_bioproject_id,ncbi_biosample_id,first_name,...,STATE,COUNTY,COUNTYA,TRACTA,U71001,U71002,U71003,UrbanThreshold,RuralThreshold,Urban_or_Rural_Census_Classification
0,0,0,555555555,MN8888888,hCoV-19/USA/MN-CDC-IBX############/YYYY,EPI_ISL_#######,test,test,test,test,...,Minnesota,Hennepin County,53,106400,1817,1817,0,100,0,Urban
